In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [4]:
def sample(law=stats.norm, params=tuple(), size=10, seed=1):
    np.random.seed(1)
    dist = law(*params)
    return {
        "law": law,
        "dist": dist,
        "seed": seed,
        "size": size,
        "sample": dist.rvs(size=size)
    }

In [35]:
tests = [
    {
        "law": stats.norm,
        "params": [(0.1,), (1.0,), (10.0,)]
    },
    {
        "law": stats.lognorm,
        "params": [(0.1,), (1.0,), (10.0,)]
    }
]

In [37]:
for test in tests:
    params = test.pop("params")
    for p in params:
        for size in [5, 10, 20]:
            for seed in [1, 2, 3]:
                x = sample(params=p, size=size, **test)
                print(x)

KeyError: 'params'